<p align="center">
    <img src="./pytorch-logo.png" width=100>
</p>

<h2 align="center">MLH Show & Tell: Introduction to PyTorch</h2>

<br/>

<div align="center">
    This notebook gives a introduction to pytorch. We'll be discussing about tensors, usage of computational graphs to calculate gradients and build a simple linear model to get an understanding of the workflow in PyTorch.
</div>

<div>
    <h3>Topics Covered</h3>
    <ol>
        <li>Tensors</li>
        <li>Computational Graphs - Autograd</li>
        <li>Datasets & Dataloaders</li>
        <li>Linear Regression</li>
        <li>Simple Neural Network</li>
    </ol>
</div>

In [ ]:
import os

import torch
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

## 1. Tensors

### 1.1 Introduction

In [ ]:
# What is a tensor?
# Difference b/w tensor and Tensor

print(torch.Tensor(), torch.tensor([1,2,3]))

a = torch.tensor([[1], [2], [3]], dtype=float, device='cpu')
b = torch.tensor([[4], [5], [6]], dtype=float, device='cpu')

a,b

In [ ]:
torch.ones((1,4,2)) # np.ones((1,4,2))
torch.zeros((1,4,2))

torch.rand(1,4)

In [ ]:
print(a.device, a.shape, a.dtype, sep='\n')

### 1.2 Coversions

In [ ]:
# Converting from array to tensor
torch.from_numpy(np.array([1,2,3,4], dtype=float))

# Converting from tensor to array
a.numpy()

# move tensor to device
a.to('cpu')

### 1.3 Tensor Operations

In [ ]:
# Multiplication

print("Multiplication operator")
print(a@b.T) # 3x1 @ 1x3 -> 3x3
print()

print("Multiplication matmul")
print(torch.matmul(a, b.T))
print()

print("Transpose")
print(a.T) # Transpose
print()

# Mean, Sum
# axis = 0 is along row and axis =1 is along column
print("Sum and Mean")
print(a.sum(axis=0), a.mean()) 
print()

print("Concat tensors")
print(torch.cat([a,b], axis=0))
print()

print(a.T) # Transpose

## 2. Computational Graphs - Autograd

For more detailed explanation on the usage of autograd, please refer to the [official documentation](https://pytorch.org/docs/stable/notes/autograd.html).

<div align='center'>
    <font size="5">$y = (a+b) * c$</font>
</div>


In [ ]:
a = torch.tensor(
    [2],
    dtype=float,
    device='cpu',
)

b = torch.tensor(
    [5],
    dtype=float,
    device='cpu',
    requires_grad=True
)

c = torch.tensor(
    [3], 
    dtype=float,
    device='cpu',
    requires_grad=True
)

In [ ]:
y = (a+b)*c
y

In [ ]:
y.backward()

In [ ]:
a.grad, b.grad, c.grad

## 3. Datasets & Data Loaders

For the sake of simplicity, we use a very small subset of a dataset. 

<b>Goal:</b> Predict the yield of apples and oranges given the temperature, rainfall and humidty.

For any given task in PyTorch, its always a good practice to great a dataset class and use a data loader to batch inputs. 

1. Create a dataset class
2. Use a data loader to batch the inputs


In [ ]:
# Features: (temp, rainfall, humidity)
inputs = np.array([
    [73, 67, 43],  [91, 88, 64], [87, 134, 58], 
    [102, 43, 37], [69, 96, 70], [73, 67, 43], 
    [91, 88, 64], [87, 134, 58], [102, 43, 37], 
    [69, 96, 70], [73, 67, 43], [91, 88, 64], 
    [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
    dtype='float32'
)

# Targets (apples, oranges)
targets = np.array([
        [56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
        [56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
        [56, 70], [81, 101], [119, 133], [22, 37], [103, 119]
    ],
    dtype='float32'
)

x_train, x_test, y_train, y_test = train_test_split(inputs, targets)

In [ ]:
len(inputs), len(targets)

### 3.1 Create Dataset

In [ ]:
class Dataset:
    
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
    
    def __len__(self,):
        return self.features.shape[0]
    
    def __getitem__(self,index):
        
        data = self.features[index]
        target = self.targets[index]
        
        return {
            "features": torch.tensor(data, dtype=torch.float,),
            "target": torch.tensor(target, dtype=torch.float,),
        }    

In [ ]:
train_dataset = Dataset(x_train, y_train)
test_dataset = Dataset(x_test, y_test)

In [ ]:
train_dataset[0], test_dataset[0]

### 3.2 Data Loader

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=2,
    num_workers=2,
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=2,
    num_workers=2,
)

## 4. Linear Regression

In [ ]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

lr_rate = 0.001
epochs = 20


def model(x):
    return x @ w.t() + b
    

In [ ]:
model(torch.from_numpy(inputs))

In [ ]:
def mse(t1, t2, ):
    diff = t1 - t2
    mse_loss = torch.sum(diff * diff) / diff.numel()
    reg = w.sum() * (0.01/(2*diff.numel()))
    
    loss = mse_loss + reg
    
    return loss 

In [ ]:
for epoch in range(epochs):
    
    epoch_loss=0 
    for sample in train_dataloader:
        
        x = sample['features']
        y = sample['target']
        
        output = model(x)
        
        loss = mse(output, y)
        loss.backward()
        
        epoch_loss += loss.item()
        
        with torch.no_grad():
            w -= w.grad * 1e-5
            b -= b.grad * 1e-5
            w.grad.zero_()
            b.grad.zero_()
    
    print(epoch, epoch_loss)
    

In [ ]:
w, b

In [ ]:
for test_inputs in test_dataloader:

    x = sample['features']
    y = sample['target']
    
    print(sample, '\n')
    print(f'True value: {y.numpy()[0]}; Prediction: {model(x).detach().numpy()[0]}')
    break
    
    

In [ ]:
len(test_dataloader)

## 5. Neural Networks

In [ ]:
import torch.nn.functional as F

In [ ]:

class NN(torch.nn.Module):
    
    def __init__(self):
        
        super().__init__()
        
        self.linear1 = nn.Linear(3, 3)
        self.act1 = torch.nn.ReLU() # Activation function
        self.linear2 = torch.nn.Linear(3, 2)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.act1(x)
        x = self.linear2(x)
        return x

In [ ]:
model.linear.weight, model.linear.bias

In [ ]:
loss_fn = F.mse_loss
opt = torch.optim.SGD(model.parameters(), lr=1e-5)


In [ ]:
for epoch in range(epochs):
    
    for sample in train_dataloader:
        
        x,y = sample['features'], sample['target']
        
        output = model(x)
        
        loss = loss_fn(output, y)
        
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    print('Training loss: ', loss.detach().numpy())
    

In [ ]:
for test_inputs in test_dataloader:

    x = sample['features']
    y = sample['target']
    
    print(sample, '\n')
    print(f'True value: {y.numpy()[0]}; Prediction: {model(x).detach().numpy()[0]}')
    break
    
    

## Common Problems

1. Always move all the inputs to the same device (`cpu` or `gpu`)
    ```python
    a.to('cpu')
    ```
2. `TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'`
    
   Make sure that `requires_grad=True`


## Personal Tips

1. Don't do lots and lots of courses without actually practicing anything.


2. Try to work on a new project every month with a new task (Regression, classification, clustering, recommendation, etc ...)


3. Particpiate in kaggle competitions. Read and understand other notebooks and methods.


4. Read review papers


-----